In [1]:
import sys
from pathlib import Path
import json

import pandas as pd
import numpy as np
import geopandas as gp
import matplotlib

DIR = Path('..')
sys.path.append(str(DIR))

import gtfs_kit as gk

DATA_DIR = DIR/'data/'

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# List feed

path = DATA_DIR/'cairns_gtfs.zip'
gk.list_feed(path)

,file_name,file_size
0,calendar_dates.txt,387
1,routes.txt,1478
2,trips.txt,143081
3,stops.txt,26183
4,agency.txt,199
5,stop_times.txt,2561019
6,shapes.txt,864694
7,calendar.txt,337


In [3]:
# Read feed and describe

path = DATA_DIR/'cairns_gtfs.zip'
feed = gk.read_feed(path, dist_units='km')
feed.describe()

,indicator,value
0,agencies,[Department of Transport and Main Roads - Tran...
1,timezone,Australia/Brisbane
2,start_date,20140526
3,end_date,20141228
4,num_routes,22
5,num_trips,1339
6,num_stops,416
7,num_shapes,54
8,sample_date,20140529
9,num_routes_active_on_sample_date,20


In [4]:
# Validate

feed.validate()

,type,message,table,rows
0,warning,Feed expired,calendar/calendar_dates,[]
1,warning,"Repeated pair (trip_id, departure_time)",stop_times,"[17710, 17722, 17730, 17732, 17736, 17739, 177..."


In [5]:
# Append shape_dist_traveled column to stop times

display(feed.stop_times.head().T)

feed = feed.append_dist_to_stop_times()
feed.stop_times.head().T

,0,1,2,3,4
trip_id,CNS2014-CNS_MUL-Weekday-00-4165878,CNS2014-CNS_MUL-Weekday-00-4165878,CNS2014-CNS_MUL-Weekday-00-4165878,CNS2014-CNS_MUL-Weekday-00-4165878,CNS2014-CNS_MUL-Weekday-00-4165878
arrival_time,05:50:00,05:50:00,05:52:00,05:54:00,05:55:00
departure_time,05:50:00,05:50:00,05:52:00,05:54:00,05:55:00
stop_id,750337,750000,750001,750002,750003
stop_sequence,1,2,3,4,5
pickup_type,0,0,0,0,0
drop_off_type,0,0,0,0,0


,0,1,2,3,4
trip_id,CNS2014-CNS_MUL-Saturday-00-4165937,CNS2014-CNS_MUL-Saturday-00-4165937,CNS2014-CNS_MUL-Saturday-00-4165937,CNS2014-CNS_MUL-Saturday-00-4165937,CNS2014-CNS_MUL-Saturday-00-4165937
arrival_time,06:16:00,06:16:00,06:18:00,06:20:00,06:21:00
departure_time,06:16:00,06:16:00,06:18:00,06:20:00,06:21:00
stop_id,750337,750000,750001,750002,750003
stop_sequence,1,2,3,4,5
pickup_type,0,0,0,0,0
drop_off_type,0,0,0,0,0
shape_dist_traveled,0,0.46864,1.19038,2.15478,2.619


In [6]:
# Choose study dates

week = feed.get_first_week()
dates = [week[4], week[6]]  # First Friday and Sunday
dates

['20140530', '20140601']

In [7]:
# Compute feed time series

trip_stats = feed.compute_trip_stats()
trip_stats.head().T
fts = feed.compute_feed_time_series(trip_stats, dates, freq='6H')
fts

indicator,num_trip_ends,num_trip_starts,num_trips,service_distance,service_duration,service_speed
datetime,,,,,,
2014-05-30 00:00:00,14.0,17.0,17.0,582.579153,12.450000,46.793506
2014-05-30 06:00:00,214.0,240.0,242.0,4954.226378,173.800000,28.505330
2014-05-30 12:00:00,248.0,253.0,281.0,5505.337390,193.216667,28.493077
2014-05-30 18:00:00,155.0,126.0,160.0,3248.284486,103.550000,31.369237
2014-05-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2014-05-31 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2014-05-31 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2014-05-31 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2014-06-01 00:00:00,0.0,0.0,0.0,26.198808,0.766667,34.172358


In [8]:
gk.downsample(fts, freq='12H')


indicator,num_trip_ends,num_trip_starts,num_trips,service_distance,service_duration,service_speed
datetime,,,,,,
2014-05-30 00:00:00,228.0,257.0,256.0,5536.805531,186.250000,29.727815
2014-05-30 12:00:00,403.0,379.0,408.0,8753.621876,296.766667,29.496648
2014-05-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2014-05-31 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2014-06-01 00:00:00,68.0,80.0,80.0,1820.286242,56.533333,32.198459
2014-06-01 12:00:00,195.0,186.0,198.0,4570.561024,141.150000,32.380879


In [9]:
# Compute feed stats for first week

feed_stats = feed.compute_feed_stats(trip_stats, week)
feed_stats

,num_stops,num_routes,num_trips,num_trip_starts,num_trip_ends,service_distance,service_duration,service_speed,peak_num_trips,peak_start_time,peak_end_time,date
0,416,20,622,622,617,13774.030583,472.600000,29.145219,39,08:16:00,08:18:00,20140526
0,416,20,622,622,617,13774.030583,472.600000,29.145219,39,08:16:00,08:18:00,20140527
0,416,20,622,622,617,13774.030583,472.600000,29.145219,39,08:16:00,08:18:00,20140528
0,416,20,622,622,617,13774.030583,472.600000,29.145219,39,08:16:00,08:18:00,20140529
0,416,22,636,636,617,14290.427407,483.016667,29.585785,39,08:16:00,08:18:00,20140530
0,416,22,437,437,415,9911.525805,310.400000,31.931462,23,17:28:00,17:37:00,20140531
0,416,14,266,266,263,6390.847266,197.683333,32.328711,17,14:31:00,14:37:00,20140601


In [10]:
# Compute route time series

rts = feed.compute_route_time_series(trip_stats, dates, freq='12H')
rts

indicator           num_trip_ends                                           \
route_id                  110-423 110N-423 111-423 112-423 113-423 120-423   
datetime                                                                     
2014-05-30 00:00:00          19.0      9.0    18.0     4.0     3.0    11.0   
2014-05-30 12:00:00          39.0      0.0    39.0    11.0     3.0    21.0   
2014-05-31 00:00:00           NaN      NaN     NaN     NaN     NaN     NaN   
2014-05-31 12:00:00           NaN      NaN     NaN     NaN     NaN     NaN   
2014-06-01 00:00:00           7.0      NaN     8.0     3.0     NaN     4.0   
2014-06-01 12:00:00          24.0      NaN    24.0     5.0     NaN    12.0   

indicator                                             ... service_speed  \
route_id            120N-423 121-423 122-423 123-423  ...      131N-423   
datetime                                              ...                 
2014-05-30 00:00:00      0.0    13.0    13.0    21.0  ...      0.000000   
2014-05-30 12:00:00      2.0    21.0    20.0    38.0  ...     24.067721   
2014-05-31 00:00:00      NaN     NaN     NaN     NaN  ...           NaN   
2014-05-31 12:00:00      NaN     NaN     NaN     NaN  ...           NaN   
2014-06-01 00:00:00      NaN     4.0     3.0     5.0  ...      0.000000   
2014-06-01 12:00:00      NaN    10.0    11.0    16.0  ...     25.503766   

indicator                                                                   \
route_id               133-423    140-423   140N-423    141-423    142-423   
datetime                                                                     
2014-05-30 00:00:00  25.034976  25.634753  46.755218  20.802002  26.021598   
2014-05-30 12:00:00  26.439007  25.832329   0.000000  20.828115  26.417789   
2014-05-31 00:00:00        NaN        NaN        NaN        NaN        NaN   
2014-05-31 12:00:00        NaN        NaN        NaN        NaN        NaN   
2014-06-01 00:00:00  27.937162        NaN        NaN        NaN  28.218140   
2014-06-01 12:00:00  27.917174        NaN        NaN        NaN  28.163040   

indicator                                                        
route_id               143-423   143W-423    150-423   150E-423  
datetime                                                         
2014-05-30 00:00:00  24.366138   0.000000  31.565566   0.000000  
2014-05-30 12:00:00  24.307749  30.331218  31.547064  34.121380  
2014-05-31 00:00:00        NaN        NaN        NaN        NaN  
2014-05-31 12:00:00        NaN        NaN        NaN        NaN  
2014-06-01 00:00:00        NaN  29.759761        NaN  33.270085  
2014-06-01 12:00:00        NaN  29.717174        NaN  33.242294  

[6 rows x 132 columns]

In [11]:
# Slice time series

inds = ['service_distance', 'service_duration', 'service_speed']
rids = ['110-423', '110N-423']

rts.loc[:, (inds, rids)]

indicator           service_distance             service_duration            \
route_id                     110-423    110N-423          110-423  110N-423   
datetime                                                                      
2014-05-30 00:00:00       677.652816  399.508778        21.016667  7.916667   
2014-05-30 12:00:00      1216.570247    0.000000        36.333333  0.000000   
2014-05-31 00:00:00              NaN         NaN              NaN       NaN   
2014-05-31 12:00:00              NaN         NaN              NaN       NaN   
2014-06-01 00:00:00       279.005147         NaN         8.066667       NaN   
2014-06-01 12:00:00       732.594405         NaN        21.266667       NaN   

indicator           service_speed             
route_id                  110-423   110N-423  
datetime                                      
2014-05-30 00:00:00     32.243592  50.464267  
2014-05-30 12:00:00     33.483585   0.000000  
2014-05-31 00:00:00           NaN        NaN  
2014-05-31 12:00:00           NaN        NaN  
2014-06-01 00:00:00     34.587415        NaN  
2014-06-01 12:00:00     34.448013        NaN

In [12]:
# Slice again by cross-section

rts.xs(rids[0], axis="columns", level=1)

indicator,num_trip_ends,num_trip_starts,num_trips,service_distance,service_duration,service_speed
datetime,,,,,,
2014-05-30 00:00:00,19.0,23.0,23.0,677.652816,21.016667,32.243592
2014-05-30 12:00:00,39.0,36.0,40.0,1216.570247,36.333333,33.483585
2014-05-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2014-05-31 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2014-06-01 00:00:00,7.0,9.0,9.0,279.005147,8.066667,34.587415
2014-06-01 12:00:00,24.0,23.0,25.0,732.594405,21.266667,34.448013


In [13]:
# Compute trip locations for every hour

rng = pd.date_range('1/1/2000', periods=24, freq='H')
times = [t.strftime('%H:%M:%S') for t in rng]
loc = feed.locate_trips(dates[0], times)
loc.head()

,trip_id,time,rel_dist,route_id,direction_id,shape_id,lon,lat
0,CNS2014-CNS_MUL-Weekday-00-4165878,06:00:00,0.152945,110-423,0,1100023,358185.343160,-1.853797e+06
1,CNS2014-CNS_MUL-Weekday-00-4165879,07:00:00,0.720587,110-423,0,1100023,364975.430686,-1.865069e+06
2,CNS2014-CNS_MUL-Weekday-00-4165880,07:00:00,0.152945,110-423,0,1100023,358185.343160,-1.853797e+06
3,CNS2014-CNS_MUL-Weekday-00-4165881,08:00:00,0.720587,110-423,0,1100023,364975.430686,-1.865069e+06
4,CNS2014-CNS_MUL-Weekday-00-4165882,08:00:00,0.202283,110-423,0,1100023,358825.454562,-1.854467e+06


In [14]:
# Build a route timetable

route_id = feed.routes['route_id'].iat[0]
feed.build_route_timetable(route_id, dates).T

,0,1,2,3,4,5,6,7,8,9,...,30963,30964,30965,30966,30967,30968,30969,30970,30971,30972
route_id,110-423,110-423,110-423,110-423,110-423,110-423,110-423,110-423,110-423,110-423,...,110-423,110-423,110-423,110-423,110-423,110-423,110-423,110-423,110-423,110-423
service_id,CNS2014-CNS_MUL-Weekday-00,CNS2014-CNS_MUL-Weekday-00,CNS2014-CNS_MUL-Weekday-00,CNS2014-CNS_MUL-Weekday-00,CNS2014-CNS_MUL-Weekday-00,CNS2014-CNS_MUL-Weekday-00,CNS2014-CNS_MUL-Weekday-00,CNS2014-CNS_MUL-Weekday-00,CNS2014-CNS_MUL-Weekday-00,CNS2014-CNS_MUL-Weekday-00,...,CNS2014-CNS_MUL-Sunday-00,CNS2014-CNS_MUL-Sunday-00,CNS2014-CNS_MUL-Sunday-00,CNS2014-CNS_MUL-Sunday-00,CNS2014-CNS_MUL-Sunday-00,CNS2014-CNS_MUL-Sunday-00,CNS2014-CNS_MUL-Sunday-00,CNS2014-CNS_MUL-Sunday-00,CNS2014-CNS_MUL-Sunday-00,CNS2014-CNS_MUL-Sunday-00
trip_id,CNS2014-CNS_MUL-Weekday-00-4165878,CNS2014-CNS_MUL-Weekday-00-4165878,CNS2014-CNS_MUL-Weekday-00-4165878,CNS2014-CNS_MUL-Weekday-00-4165878,CNS2014-CNS_MUL-Weekday-00-4165878,CNS2014-CNS_MUL-Weekday-00-4165878,CNS2014-CNS_MUL-Weekday-00-4165878,CNS2014-CNS_MUL-Weekday-00-4165878,CNS2014-CNS_MUL-Weekday-00-4165878,CNS2014-CNS_MUL-Weekday-00-4165878,...,CNS2014-CNS_MUL-Sunday-00-4166102,CNS2014-CNS_MUL-Sunday-00-4166102,CNS2014-CNS_MUL-Sunday-00-4166102,CNS2014-CNS_MUL-Sunday-00-4166102,CNS2014-CNS_MUL-Sunday-00-4166102,CNS2014-CNS_MUL-Sunday-00-4166102,CNS2014-CNS_MUL-Sunday-00-4166102,CNS2014-CNS_MUL-Sunday-00-4166102,CNS2014-CNS_MUL-Sunday-00-4166102,CNS2014-CNS_MUL-Sunday-00-4166102
trip_headsign,The Pier Cairns Terminus,The Pier Cairns Terminus,The Pier Cairns Terminus,The Pier Cairns Terminus,The Pier Cairns Terminus,The Pier Cairns Terminus,The Pier Cairns Terminus,The Pier Cairns Terminus,The Pier Cairns Terminus,The Pier Cairns Terminus,...,Palm Cove,Palm Cove,Palm Cove,Palm Cove,Palm Cove,Palm Cove,Palm Cove,Palm Cove,Palm Cove,Palm Cove
direction_id,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
block_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shape_id,1100023,1100023,1100023,1100023,1100023,1100023,1100023,1100023,1100023,1100023,...,1100016,1100016,1100016,1100016,1100016,1100016,1100016,1100016,1100016,1100016
arrival_time,05:50:00,05:50:00,05:52:00,05:54:00,05:55:00,05:57:00,05:59:00,06:00:00,06:01:00,06:02:00,...,23:50:00,23:51:00,23:52:00,23:54:00,23:54:00,23:56:00,23:59:00,24:01:00,24:02:00,24:04:00
departure_time,05:50:00,05:50:00,05:52:00,05:54:00,05:55:00,05:57:00,05:59:00,06:00:00,06:01:00,06:02:00,...,23:50:00,23:51:00,23:52:00,23:54:00,23:54:00,23:56:00,23:59:00,24:01:00,24:02:00,24:04:00
stop_id,750337,750000,750001,750002,750003,750004,750005,750006,750007,750008,...,750344,750343,750342,750036,750037,750038,750339,750039,750040,750338


In [15]:
# Compute screen line counts

path = DATA_DIR/'cairns_screen_lines.geojson'
lines = gp.read_file(path)

display(lines)

feed.compute_screen_line_counts(lines, dates)

,geometry
0,"LINESTRING (145.77310 -16.91738, 145.77358 -16..."
1,"LINESTRING (145.76706 -16.91638, 145.76752 -16..."


,date,trip_id,route_id,route_short_name,shape_id,screen_line_id,crossing_direction,crossing_distance,crossing_time
0,20140530,CNS2014-CNS_MUL-Weekday-00-4165878,110-423,110,1100023,sl1,1,30.299004,06:40:55
1,20140530,CNS2014-CNS_MUL-Weekday-00-4165878,110-423,110,1100023,sl0,1,31.279921,06:44:29
2,20140530,CNS2014-CNS_MUL-Weekday-00-4165879,110-423,110,1100023,sl1,1,30.299004,07:10:55
3,20140530,CNS2014-CNS_MUL-Weekday-00-4165879,110-423,110,1100023,sl0,1,31.279921,07:14:29
4,20140530,CNS2014-CNS_MUL-Weekday-00-4165880,110-423,110,1100023,sl1,1,30.299004,07:40:55
...,...,...,...,...,...,...,...,...,...
883,20140601,CNS2014-CNS_MUL-Sunday-00-4172783,131-423,131,1310022,sl1,-1,2.884846,20:37:16
884,20140601,CNS2014-CNS_MUL-Sunday-00-4172784,131-423,131,1310022,sl0,-1,1.496863,22:02:50
885,20140601,CNS2014-CNS_MUL-Sunday-00-4172784,131-423,131,1310022,sl1,-1,2.884846,22:07:16
886,20140601,CNS2014-CNS_MUL-Sunday-00-4172790,131N-423,131N,131N0007,sl0,-1,1.496863,23:02:50


In [16]:
# Map routes

rids = feed.routes.route_id.loc[2:4]
feed.map_routes(rids, include_stops=True)
